# convert images to jpg

In [85]:
Path = './yalefaces/'

Path2 = './yaleface_jpg/'

files = os.listdir(Path)
files2 = os.listdir(Path2)
images = []

for name in files:
    temp  = Image.open(Path+name).save('./yaleface_jpg/'+name+'.jpg')


for name in files2:
    temp = cv2.imread(Path2+name)
    images.append(temp)

In [2]:
#importing required library

import os
import cv2
import numpy as np
import math
from PIL import Image

In [3]:
#initializing global parameters

# folder_tr = "yaleface_jpg/tr"
# folder_te = "yaleface_jpg/te"
folder_tr = "tr"
folder_te = "te"
total_tr = len(next(os.walk(os.getcwd()+"/"+folder_tr))[2])
total_te = len(next(os.walk(os.getcwd()+"/"+folder_te))[2])
dim = 200

# training starts from here

In [4]:
# input image maytix -> column
# matrix L: all images column vise

f = np.empty(shape=(dim*dim,total_tr), dtype='float64')
i = 0
for filename in os.listdir(os.getcwd()+"/"+folder_tr):
    #Read a image 
    image = cv2.imread(folder_tr+"/"+filename)
    #resize image
    resized = cv2.resize(image, (dim,dim), interpolation = cv2.INTER_AREA)
    #Convert it into grayscale image
    gray_image = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)
    #flattening an image
    gray_vector = np.array(gray_image, dtype='float64').flatten()
    f[:, i] = gray_vector[:] 
    i = i + 1

In [5]:
#Mean face
mean_face = np.sum(f, axis=1) /total_tr

#subtract mean face
for i in range(total_tr):
    f[:,i] -= mean_face[:]

In [6]:
# SVD
u, s, vh = np.linalg.svd(f, full_matrices=False)

In [7]:
# Creating emplty array to store f*u
fu = np.empty(shape = (u.shape[0]*u.shape[1], u.shape[1]),  dtype=np.int8)

# Creating temp array
temp = np.empty(shape = (u.shape[0], u.shape[1]),  dtype=np.int8)

In [8]:
# loop over number of faces f, for f*u, store the answer of array by flattning it in fu array

for i in range(f.shape[1]):
    
    for c in range(u.shape[1]):    
        temp[:,c] = f[:,i] * u[:,c]
    
    tempF = np.array(temp, dtype='int8').flatten()
    
    fu[:, i] = tempF[:]

# testing starts from here

In [9]:
tt = np.empty(shape = (u.shape[0], u.shape[1]),  dtype=np.int8)

i = 0
wrong = 0
for filename in os.listdir(os.getcwd()+"/"+folder_te):
    print("input face: ",filename)
    # read image
    test = cv2.imread(folder_te+"/"+filename)   
    # resize
    test = cv2.resize(test, (dim,dim), interpolation = cv2.INTER_AREA)
    # convert to gray scale
    test= cv2.cvtColor(test, cv2.COLOR_BGR2GRAY)
    # flatten image
    test = np.array(test, dtype='float64').flatten()  
    # subtract mean
    test -= mean_face     

    # matrix: tt (test image as column vector)
    tt[:, i] = test[:]
    i=i+1

input face:  subject01_glasses.jpg


In [10]:
# test image * u (u of SVD)

# Creating emplty array to store f*u
t = np.empty(shape = (u.shape[0], u.shape[1]),  dtype=np.int8)
    
for col in range(u.shape[1]):    
    t[:,col] = tt[:,0] * u[:,col]
    
# flatten whole tt*u matrix
tF = np.array(t, dtype='int8').flatten()

In [11]:
# fu - tf

fu_tu = np.empty(shape = (u.shape[0]*u.shape[1], u.shape[1]))

for col in range(u.shape[1]):
    fu_tu[:,col] = fu[:,col] - tF[:]

In [12]:
# find norm

from numpy import linalg as LA

ans = np.empty(shape=(u.shape[1],))

for c in range(fu_tu.shape[1]):    
    ans[c] = np.linalg.norm(fu_tu[:,c])

In [16]:
# shorting norm values

temp_ans = np.empty(shape=(u.shape[1],))
temp=np.copy(ans)

temp.sort()
check = temp[0]
print("norm: ",check)

print(ans)
print(temp)

norm:  103.261803199
[ 103.2618032   199.73983078  297.2877394   325.81896814  273.78823934
  211.45685139  267.17410054  238.40721466  180.0166659   219.34447793
  298.44094893  268.26852219  390.75439857  262.80791464  213.11499243
  276.28789333  225.09997779  233.54871012  255.97265479  330.12270446
  314.591799    314.09075122  288.74556274  238.04831442]
[ 103.2618032   180.0166659   199.73983078  211.45685139  213.11499243
  219.34447793  225.09997779  233.54871012  238.04831442  238.40721466
  255.97265479  262.80791464  267.17410054  268.26852219  273.78823934
  276.28789333  288.74556274  297.2877394   298.44094893  314.09075122
  314.591799    325.81896814  330.12270446  390.75439857]


In [14]:
# find image index at the minimum norm
index=0

for i in range(ans.shape[0]):
    if check == ans[i]:
        print("image index:",i)
        index=i
        break

image index: 0


In [15]:
# print output image name
i = 0
index
for filename in os.listdir(os.getcwd()+"/"+folder_tr):
    
    if index == i:
        print("recognized face: ",filename)
        break
        
    else:
        i=i+1

recognized face:  subject01_glasses.jpg
